In [1]:
#-*- coding: UTF-8 -*-
from __future__ import division
import os,sys
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from PIL import Image
import tensorflow as tf

import random
import time
br='\n'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
class Data(object):
    def __init__ (self):
        self._epochs_completed=0
        self._index_in_epoch=0
        
        train_x=[]
        train_y=[]
        
        image_files=os.listdir('data12/trainingDigits')
        for image_file in image_files:
            image=[]
            fr=open('data12/trainingDigits'+'/'+image_file)
            for line in fr.readlines():
                line =line.strip()
                list_line=[]
                for i in range(32):
                    list_line.append(int(line[i]))
                image.append(list_line)    
            train_x.append(np.asarray(image))
            
            label=np.zeros(10)
            label[int(image_file.split('_')[0])]=1
            train_y.append(label)
            
        self.train_images=train_x
        self.train_labels=train_y
        data=list(zip(self.train_images,self.train_labels))
        random.shuffle(data)
        self.train_images[:],self.train_labels[:]=zip(*data)
        self._num_examples=len(self.train_images)
    def batch_inter(self,batch_size):
        self._num_examples=len(self.train_images)
        start=self._index_in_epoch
        self._index_in_epoch+=batch_size
        
        if self._index_in_epoch>self._num_examples:
            self._epochs_completed+=1
            data=list(zip(self.train_images,self.train_labels))
            random.shuffle(data)
            self.train_images[:],self.train_labels[:]=zip(*data)
            start=0
            self._index_in_epoch=batch_size
            assert batch_size<=self._num_examples
        end=self._index_in_epoch
        return self.train_images[start:end],self.train_labels[start:end]
    
    def test_data(self):
        test_x=[]
        test_y=[]
        
        image_files=os.listdir('data12/testDigits')
        for image_file in image_files:
            image=[]
            fr=open('data12/testDigits'+'/'+image_file)
            for line in fr.readlines():
                line=line.strip()
                list_line=[]
                for i in range(32):
                    list_line.append(int(line[i]))
                image.append(list_line)
            test_x.append(np.asarray(image))
            label=np.zeros(10)
            label[int(image_file.split('_')[0])]=1
            test_y.append(label)
        
        self.test_images=test_x
        self.test_labels=test_y
        data=list(zip(self.test_images,self.test_labels))
        random.shuffle(data)
        self.test_images[:],self.test_labels[:]=zip(*data)
        
        return self.test_images,self.test_labels
    
    def __len__(self):
        return self._num_examples
    

In [37]:
data=Data()
test_images,test_labels=data.test_data()

In [38]:
print(test_images[0][0])
print(test_images[0][1])
print('训练样本个数：%d'%(len(data)))
print("测试样本个数:%d"%(len(test_images)))

[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
训练样本个数：1498
测试样本个数:434


In [49]:
class Model_CNN_digit(object):
    def __init__(self):
        def weight_variable(shape):
            initial=tf.truncated_normal(shape,stddev=0.1)
            return tf.Variable(initial)
        def bias_variable(shape):
            initial=tf.constant(0.1,shape=shape)
            return tf.Variable(initial)
        def conv2d(x,W):
            return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
            return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
        
        def max_pool_2x2(x):
            return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        
        x=tf.placeholder(tf.float32,[None,32,32])
        y=tf.placeholder(tf.float32,[None,10])
        x_image=tf.reshape(x,[-1,32,32,1])
        
        W_convl=weight_variable([5,5,1,32])
        b_convl=bias_variable([32])
        h_convl=tf.nn.relu(conv2d(x_image,W_convl)+b_convl)
        h_pool1=max_pool_2x2(h_convl)
        
        W_conv2=weight_variable([5,5,32,64])
        b_conv2=bias_variable([64])
        h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
        h_pool2=max_pool_2x2(h_conv2)
        
        W_fc1=weight_variable([8*8*64,1024])
        b_fc1=bias_variable([1024])
        h_pool2_flat=tf.reshape(h_pool2,[-1,8*8*64])
        h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
        h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)
        
        W_fc2=weight_variable([1024,10])
        b_fc2=bias_variable([10])
        pred=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
        
        cost=tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred),reduction_indices=1))
        optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)
        
        correct_perdiction=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
        accuracy=tf.reduce_mean(tf.cast(correct_perdiction,tf.float32))
        
        self.x=x
        self.y=y
        self.pred=pred
        self.cost=cost
        self.accuracy=accuracy
        self.optimizer=optimizer
        
    def train_model(self,sess,batch_xs,batch_ys):
        feed_dict={
            self.x:batch_xs,
            self.y:batch_ys
        }
        to_return={
            'train_op':self.optimizer,
            'loss':self.cost,
            'acc':self.accuracy
        }
        return sess.run(to_return,feed_dict)
    
    def test_model(self,sess,batch_xs):
        feed_dict={
            self.x:batch_xs
        }
        to_return={
            'pred':self.pred
        }
        return sess.run(to_return,feed_dict)
    
    def eval_model(self,sess,batch_xs,batch_ys):
        correct_prediction=tf.equal(tf.argmax(self.pred,1),tf.argmax(self.y,1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        feed_dict={
            self.x:batch_xs,
            self.y:batch_ys
        }
        to_return={
            'acc':accuracy
        }
        return sess.run(to_return,feed_dict)
        
        

In [50]:
learning_rate=1e-4
training_epochs=50
batch_size=100
display_step=1
keep_prob=0.75

In [51]:
model=Model_CNN_digit()

os.environ['CUDA_VISIBLE_DEVICES']='0'
config=tf.ConfigProto()
config.gpu_options.allow_growth=True

In [55]:
with tf.Session() as sess:
    init=tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost=0
        avg_cost_list=[]
        avg_acc=0.
        total_batch=int(len(data)/batch_size)
        
        for i in range(total_batch):
            batch_xs,batch_ys=data.batch_inter(batch_size)
            
            results=model.train_model(sess,batch_xs,batch_ys)
            c=results['loss']
            ac=results['acc']
            
            avg_cost_list.append(c)
            
            avg_cost+=c/total_batch
            avg_acc+=ac/total_batch
            
        if (epoch+1)%display_step==0:
            print("CNN Epoch:",'%04d'%(epoch+1),"cost:","{:.9f}".format(avg_cost),"accuracy=","{:.9f}".format(avg_acc))
            print("Optimization Finished!")
            test_xs,test_ys=data.test_data()
            
            acc=model.eval_model(sess,test_xs,test_ys)
            print("the number of test data:",len(test_xs))
            print("Accuracy:",acc)
            

CNN Epoch: 0001 cost: 8.489876168 accuracy= 0.183571428
Optimization Finished!
the number of test data: 434
Accuracy: {'acc': 0.3202765}
CNN Epoch: 0002 cost: 3.246395503 accuracy= 0.452142852
Optimization Finished!
the number of test data: 434
Accuracy: {'acc': 0.5760369}
CNN Epoch: 0003 cost: 1.715952814 accuracy= 0.660714277
Optimization Finished!
the number of test data: 434
Accuracy: {'acc': 0.6797235}
CNN Epoch: 0004 cost: 1.020967701 accuracy= 0.747857136
Optimization Finished!
the number of test data: 434
Accuracy: {'acc': 0.7903226}
CNN Epoch: 0005 cost: 0.719884645 accuracy= 0.807142858
Optimization Finished!
the number of test data: 434
Accuracy: {'acc': 0.8110599}
CNN Epoch: 0006 cost: 0.560259632 accuracy= 0.846428573
Optimization Finished!
the number of test data: 434
Accuracy: {'acc': 0.84101385}
CNN Epoch: 0007 cost: 0.453952519 accuracy= 0.880000004
Optimization Finished!
the number of test data: 434
Accuracy: {'acc': 0.86175114}
CNN Epoch: 0008 cost: 0.323519762 accur

In [ ]:
class Data(object):
    def __init__ (self):
        self._epochs_completed=0
        self._index_in_epoch=0
        classfile_names_train=os.listdir('data12/traindata')
        train_x=[]
        train_y=[]
        for classfile_name in classfile_names_train:
            trainfile_names=os.listdir('data12/traindata/'+classfile_name)
            for trainfile_name in trainfile_names:
                image=image.resize((28,28))
                image_array=(np.asarray(image,dtype='float32')/255)
                train_x.append(image_array)
                label=np.zeros(43)
                label[int(classfile_name)]=1
                train_y.append(label)
        self.train_images=train_x
        self.train_labels=train_y
        data=list(zip(self.train_images,self.train_labels))
        random.shuffle(data)
        self.train_images[:],self.train_labels[:]=zip(*data)
        self._nim_examples=len(self.train_images)
        
    def batch_inter(self,batch_size):
        self._num_examples=len(self.train_images)
        start=self._index_in_epoch
        self._index_in_epoch+=batch_size
        if self._index_in_epoch>self._num_examples:
            self._epochs_completed+=1
            data=list(zip(self.train_images,self.train_labels))
            random.shuffle(data)
            self.train_images[:],self.train_labels[:]=zip(*data)
            start=0
            self._index_in_epoch=batch_size
            assert batch_size<=self._num_examples
        end=self._index_in_epoch
        return self.train_images[start:end],self.train_labels[start:end]
    
    def test_data(self):
        test_x=[]
        test_y=[]
        classfile_names_test=os.listdir('data12/testdata')
        for classfile_name in classfile_names_test:
            testfile_names=os.listdir('data12/testdata/'+classfile_name)
            for testfile_name in testfile_names:
                image=Imaga.open('data12/testdata/'+classfile_name+'/'+testfile_name)
                iamge=image.resize((28,28))
                image_array=(np.asarray(image,dtype='float32')/255)
                test_x.append(image_array)
                label=np.zeros(43)
                label[int(classfile_name)]=1
                test_y.append(label)
        self.test_images=test_x
        self.test_labels=test_y
        data=list(zip(self.test_images,self.test_labels))
        random.shuffle(data)
        self.test_images[:],self.test_labels[:]=zip(*data)
        
        return self.test_images,self.test_labels
    
    def __len__(self):
        return self._num_examples
                
        